
# Welcome to Feature Engineering! #

# What is Feature Engineering? #

We don't have to use our data just as it's given to us. Can we make it more informative?

Feature engineering is something we encounter all the time in day to day life. Consider wind chill, for instance. 

# The Practice of Feature Engineering #

Several things to consider when coming up with a feature engineering strategy.

The technique should be adapted to the task.

Need to be careful of data leakage.

# Example - Using Domain Knowledge #

The most common kind of feature engineering is what we might call **domain knowledge**. Is there information we suspect should be useful?

Now let's define a transformer.

In this case, we dropped the original feature from the dataset. Sometimes it could be useful to leave it in.

Now let's engineer some features.

And train!

# Your Turn #